# Benchmarking against PhageDPO and DePP

1. First download PhageDPO from the [Galaxy framework](https://toolshed.g2.bx.psu.edu/repository?repository_id=77543962c4205a70) and get all the necessary files. This notebook can be put right in the same folder.
2. Download [the CLI code](https://github.com/DamianJM/Depolymerase-Predictor/tree/main/DePP_CLI) for DePP.

### Libraries

In [6]:
import os
import sys
import pickle
import subprocess
from Bio.Seq import Seq
from Bio import SeqIO
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import warnings
warnings.filterwarnings("ignore")
# PhageDPO
from local_ctd import CalculateCTD
from local_AAComposition import CalculateDipeptideComposition
# DePP
from DePP_processing import read_fasta, calculate_protein_parameters, generate_dataframe, save_dataframe
from DePP_prediction import train_model, predict_depolymerases

## PhageDPO in a notebook

In [20]:
class PDPOPrediction:

    def __init__(self, folder='location', mdl='', seq_file='fasta_file.fasta', ttable=11):
        """
        Initialize PhageDPO prediction.
        :param folder: data path
        :param mdl: ml model, in this case ANN or SVM
        :param seq_file: fasta file
        :param ttable: Translational table. By default, The Bacterial, Archaeal and Plant Plastid Code Table 11
        """
        self.records = []
        self.data = {}
        self.df_output = None
        self.seqfile = seq_file
        self.__location__ = os.path.realpath(os.path.join(os.getcwd(), folder))

        with open(os.path.join(self.__location__, mdl), 'rb') as m:
            self.model0 = pickle.load(m)
            self.model = self.model0.named_steps['clf']
            self.scaler = self.model0.named_steps['scl']
            self.selectk = self.model0.named_steps['selector']
            self.name = 'model'

        for seq in SeqIO.parse(os.path.join(self.__location__, self.seqfile), 'fasta'):
            record = []
            DNA_seq = seq.seq
            AA_seq = DNA_seq.translate(table=ttable)
            descr_seq = seq.description.replace(' ', '')
            self.data[descr_seq] = [DNA_seq._data, AA_seq._data]
            record.append(seq.description)
            record.append(str(DNA_seq))
            record.append(str(AA_seq))
            self.records.append(record)

        columns = ['ID', 'DNAseq', 'AAseq']
        self.df = pd.DataFrame(self.records, columns=columns)
        #self.df = self.df.set_index('ID')
        self.df.update(self.df.DNAseq[self.df.DNAseq.apply(type) == list].str[0])
        self.df.update(self.df.AAseq[self.df.AAseq.apply(type) == list].str[0])

    def Datastructure(self):
        """
        Create dataset with all features
        """
        def count_orf(orf_seq):
            """
            Function to count open reading frames
            :param orf_seq: sequence to analyze
            :return: dictionary with open reading frames
            """
            dic = {'DNA-A': 0, 'DNA-C': 0, 'DNA-T': 0, 'DNA-G': 0, 'DNA-GC': 0}
            for letter in range(len(orf_seq)):
                for k in range(0, 4):
                    if str(orf_seq[letter]) in list(dic.keys())[k][-1]:
                        dic[list(dic.keys())[k]] += 1
            dic['DNA-GC'] = ((dic['DNA-C'] + dic['DNA-G']) / (
                    dic['DNA-A'] + dic['DNA-C'] + dic['DNA-T'] + dic['DNA-G'])) * 100
            return dic

        def count_aa(aa_seq):
            """
            Function to count amino acids
            :param aa_seq: sequence to analyze
            :return: dictionary with amino acid composition
            """
            dic = {'G': 0, 'A': 0, 'L': 0, 'V': 0, 'I': 0, 'P': 0, 'F': 0, 'S': 0, 'T': 0, 'C': 0,
                   'Y': 0, 'N': 0, 'Q': 0, 'D': 0, 'E': 0, 'R': 0, 'K': 0, 'H': 0, 'W': 0, 'M': 0}
            for letter in range(len(aa_seq)):
                if aa_seq[letter] in dic.keys():
                    dic[aa_seq[letter]] += 1
            return dic

        def sec_st_fr(aa_seq):
            """
            Function to analyze secondary structure. Helix, Turn and Sheet
            :param aa_seq: sequence to analyze
            :return: dictionary with composition of each secondary structure
            """
            st_dic = {'Helix': 0, 'Turn': 0, 'Sheet': 0}
            stu = ProteinAnalysis(aa_seq).secondary_structure_fraction()
            st_dic['Helix'] = stu[0]
            st_dic['Turn'] = stu[1]
            st_dic['Sheet'] = stu[2]
            return st_dic


        self.df_output = self.df.copy()
        self.df_output.drop(['DNAseq', 'AAseq'], axis=1, inplace=True)
        dna_feat = {}
        aa_len = {}
        aroma_dic = {}
        iso_dic = {}
        aa_content = {}
        st_dic_master = {}
        CTD_dic = {}
        dp = {}
        self.df1 = self.df[['ID']].copy()
        self.df.drop(['ID'], axis=1, inplace=True)
        for i in range(len(self.df)):
            i_name = self.df.index[i]
            dna_feat[i] = count_orf(self.df.iloc[i]['DNAseq'])
            aa_len[i] = len(self.df.iloc[i]['AAseq'])
            aroma_dic[i] = ProteinAnalysis(self.df.iloc[i]['AAseq']).aromaticity()
            iso_dic[i] = ProteinAnalysis(self.df.iloc[i]['AAseq']).isoelectric_point()
            aa_content[i] = count_aa(self.df.iloc[i]['AAseq'])
            st_dic_master[i] = sec_st_fr(self.df.iloc[i]['AAseq'])
            CTD_dic[i] = CalculateCTD(self.df.iloc[i]['AAseq'])
            dp[i] = CalculateDipeptideComposition(self.df.iloc[i]['AAseq'])
        for j in self.df.index:
            self.df.loc[j, dna_feat[j].keys()] = dna_feat[j].values() #dic with multiple values
            self.df.loc[j, 'AA_Len'] = int(aa_len[j]) #dic with one value
            self.df.loc[j, 'Aromaticity'] = aroma_dic[j]
            self.df.loc[j, 'IsoelectricPoint'] = iso_dic[j]
            self.df.loc[j, aa_content[j].keys()] = aa_content[j].values()
            self.df.loc[j, st_dic_master[j].keys()] = st_dic_master[j].values()
            self.df.loc[j, CTD_dic[j].keys()] = CTD_dic[j].values()
            self.df.loc[j, dp[j].keys()] = dp[j].values()
        self.df.drop(['DNAseq', 'AAseq'], axis=1, inplace=True)

    def Prediction(self, outname):
        """
        Predicts the percentage of each CDS being depolymerase.
        :return: model prediction
        """
        scores = self.model0.predict_proba(self.df.iloc[:, :])
        pos_scores = np.empty((self.df.shape[0], 0), float)
        for x in scores:
            pos_scores = np.append(pos_scores, round(x[1]*100))
        self.df_output.reset_index(inplace=True)
        self.df_output.rename(columns={'index': 'CDS'}, inplace=True)
        self.df_output['CDS'] += 1
        self.df_output['PhageDPO_score'] = pos_scores
        #self.df_output = self.df_output[self.df_output['PhageDPO_score'] > 50] 
        #self.df_output.to_csv(outname+'.csv', index=False)

#### Steps
1. set the path to the project
2. set the path to the fasta file you want to run
3. run the two code cells below

In [46]:
# set paths and model
data_path = '/Users/dimi/Desktop/phagedpo'
fasta_path = '/Users/dimi/Desktop/pires_dna_sequences/CDS_NC_019917.1.fasta'
model = 'svm1495'

In [52]:
# run phageDPO
PDPO = PDPOPrediction(data_path, model, fasta_path)
PDPO.Datastructure()
PDPO.Prediction(outname='thisfastaout')

#### Do multiple fastas in one go

In [35]:
# set paths and model
code_path = '/Users/dimi/Documents/GitHub/funstuff/phagedpo_tweak'
benchmarkdf = pd.read_csv('/Users/dimi/Documents/GitHub/PhageDEPOdetection/data/Benchmarking/benchmark_dataframe.csv')
model = 'svm1495'

In [29]:
bar = tqdm(total=benchmarkdf.shape[0])
scores_phageDPO = []
for i, protein_id in enumerate(benchmarkdf['protein_id']):
    # get dna seq and make fasta
    dna_sequence = benchmarkdf['dna_seq'][i]
    f = open(code_path+'/temp_fasta.fasta', 'w')
    f.write('>'+protein_id+'\n'+dna_sequence+'\n')
    f.close()
    # make phageDPO prediction
    PDPO = PDPOPrediction(code_path, model, code_path+'/temp_fasta.fasta')
    PDPO.Datastructure()
    PDPO.Prediction(outname=protein_id)
    if len(PDPO.df_output['PhageDPO_score']) > 1:
        print('oops, somethings wrong')
    score = PDPO.df_output['PhageDPO_score'][0]
    scores_phageDPO.append(score)
    # remove fasta
    os.remove(code_path+'/temp_fasta.fasta')
    # update progress
    bar.update()
bar.close

  0%|          | 0/17851 [00:00<?, ?it/s]

<bound method tqdm_notebook.close of <tqdm.notebook.tqdm_notebook object at 0x164b5d100>>

In [38]:
benchmarkdf = pd.concat([benchmarkdf, pd.DataFrame({'scores_phageDPO': scores_phageDPO})], axis=1)
benchmarkdf.to_csv('/Users/dimi/Documents/GitHub/PhageDEPOdetection/data/Benchmarking/benchmark_dataframe.csv', index=False)

## DePP in a notebook

In [40]:
# train DePP model with original training data
training_set_path = code_path + '/DePP_TrainingSet.csv'
model = train_model(training_set_path)

In [64]:
# do the predictions for the benchmark data
benchmarkdf = pd.read_csv('/Users/dimi/Documents/GitHub/PhageDEPOdetection/data/Benchmarking/benchmark_dataframe.csv')
bar = tqdm(total=benchmarkdf.shape[0])
scores_DePP = []
for i, protein_id in enumerate(benchmarkdf['protein_id']):
    # get protein seq and make fasta
    protein_sequence = benchmarkdf['protein_seq'][i]
    if protein_sequence.count('X') > 0:
        scores_DePP.append(0) # -> can't predict with their method, so we take it as 0
    else:
        f = open(code_path+'/temp_fasta.fasta', 'w')
        f.write('>'+protein_id+'\n'+protein_sequence+'\n')
        f.close()
        # make DePP prediction
        protein_records = read_fasta(code_path+'/temp_fasta.fasta')
        protein_parameters = [calculate_protein_parameters(protein_record) for protein_record in protein_records]
        protein_df = generate_dataframe(protein_parameters)
        predictions = predict_depolymerases(protein_df, model)
        if len(predictions['Probability_DePol']) > 1:
            print('oops, somethings wrong')
        score = predictions['Probability_DePol'][0]
        scores_DePP.append(score)
        # remove fasta
        os.remove(code_path+'/temp_fasta.fasta')
    # update progress
    bar.update()
bar.close

  0%|          | 0/17851 [00:00<?, ?it/s]

<bound method tqdm_notebook.close of <tqdm.notebook.tqdm_notebook object at 0x165c00850>>

In [65]:
benchmarkdf = pd.concat([benchmarkdf, pd.DataFrame({'scores_DePP': scores_DePP})], axis=1)
benchmarkdf.to_csv('/Users/dimi/Documents/GitHub/PhageDEPOdetection/data/Benchmarking/benchmark_dataframe.csv', index=False)

## Compute metrics

In [79]:
from sklearn.metrics import f1_score, confusion_matrix, recall_score, precision_score, matthews_corrcoef
# phageDPO
preds_phageDPO = benchmarkdf['scores_phageDPO'] > 50
preds_phageDPO = preds_phageDPO.astype(int)
tn, fp, fn, tp = confusion_matrix(benchmarkdf['label'], preds_phageDPO).ravel()
sensitivity = tp / (tp+fn)
specificity = tn / (tn+fp)
print('PhageDPO metrics')
print('F1: ', round(f1_score(benchmarkdf['label'], preds_phageDPO), 3))
print('R: ', round(recall_score(benchmarkdf['label'], preds_phageDPO), 3))
print('P: ', round(precision_score(benchmarkdf['label'], preds_phageDPO), 3))
print('Se: ', round(sensitivity, 3))
print('Sp: ', round(specificity, 3))
print('MCC: ', round(matthews_corrcoef(benchmarkdf['label'], preds_phageDPO), 3))
print()

# DePP
preds_depp = benchmarkdf['scores_DePP'] > 0.5
preds_depp = preds_depp.astype(int)
tn, fp, fn, tp = confusion_matrix(benchmarkdf['label'], preds_depp).ravel()
sensitivity = tp / (tp+fn)
specificity = tn / (tn+fp)
print('DePP metrics')
print('F1: ', round(f1_score(benchmarkdf['label'], preds_depp), 3))
print('R: ', round(recall_score(benchmarkdf['label'], preds_depp), 3))
print('P: ', round(precision_score(benchmarkdf['label'], preds_depp), 3))
print('Se: ', round(sensitivity, 3))
print('Sp: ', round(specificity, 3))
print('MCC: ', round(matthews_corrcoef(benchmarkdf['label'], preds_depp), 3))

PhageDPO metrics
F1:  0.202
R:  0.858
P:  0.114
Se:  0.858
Sp:  0.942
MCC:  0.301

DePP metrics
F1:  0.122
R:  0.948
P:  0.065
Se:  0.948
Sp:  0.881
MCC:  0.232


For PhageDPO, the precision is very low (which means lots of False Positives if we take Pires as ground truth), and as a result the F1 score is also very low. For DePP, recall/sensitivity better but rest of metrics even worse than PhageDPO!